In [1]:
import pandas as pd
import numpy as np
from multiprocessing import cpu_count, Pool

from sys import path
path.append('../py/')
import utils

In [2]:
ll ../output/

total 8505900
-rw-rw-r-- 1 Kazuki 464402276 Oct 14 22:48 1015-1.csv.gz
-rw-rw-r-- 1 Kazuki 496058539 Oct 15 03:49 1015-2.csv.gz
-rw-rw-r-- 1 Kazuki 496517344 Oct 15 05:54 1015-2_norm.csv.gz
-rw-rw-r-- 1 Kazuki 495961439 Oct 15 09:02 1015-3.csv.gz
-rw-rw-r-- 1 Kazuki 495234941 Oct 15 12:20 1015-4.csv.gz
-rw-rw-r-- 1 Kazuki 465782097 Oct 16 12:15 1016-1.csv.gz
-rw-rw-r-- 1 Kazuki 496235989 Oct 22 16:31 1022-1.csv.gz
-rw-rw-r-- 1 Kazuki 301176865 Nov  2 14:00 1022-1_Giba-post2-1102.csv.gz
-rw-rw-r-- 1 Kazuki 328807757 Nov  2 18:38 1022-1_Giba-post2-1103.csv.gz
-rw-rw-r-- 1 Kazuki 299206323 Oct 29 02:41 1022-1_Giba-post2.csv.gz
-rw-rw-r-- 1 Kazuki 466073193 Oct 28 18:22 1022-1_Giba-post.csv.gz
-rw-rw-r-- 1 Kazuki 465882787 Oct 29 02:55 1022-1_yuval-post2.csv.gz
-rw-rw-r-- 1 Kazuki 465882786 Oct 28 18:39 1022-1_yuval-post.csv.gz
-rw-rw-r-- 1 Kazuki 495912794 Oct 26 07:51 1026-1.csv.gz
-rw-rw-r-- 1 Kazuki 327808317 Oct 26 11:08 1026-2.csv.gz
-rw-rw-r-- 1 Kazuki 492728101 Oct 27 02:58 1027-1.

In [3]:
sub = pd.read_csv('../output/1103-2.csv.gz')

In [4]:
y_pred = sub.iloc[:, 1:].values

In [5]:
y_pred

array([[0.00000000e+00, 2.23553094e-04, 0.00000000e+00, ...,
        0.00000000e+00, 3.35000973e-03, 1.70000000e-01],
       [0.00000000e+00, 5.99053733e-03, 0.00000000e+00, ...,
        0.00000000e+00, 1.40636104e-03, 1.70000000e-01],
       [0.00000000e+00, 4.72240210e-03, 0.00000000e+00, ...,
        0.00000000e+00, 1.69739055e-03, 1.70000000e-01],
       ...,
       [0.00000000e+00, 3.49196621e-03, 0.00000000e+00, ...,
        0.00000000e+00, 5.49419744e-03, 1.70000000e-01],
       [0.00000000e+00, 5.11314763e-01, 0.00000000e+00, ...,
        0.00000000e+00, 1.27451435e-03, 1.70000000e-01],
       [0.00000000e+00, 2.76968192e-02, 0.00000000e+00, ...,
        0.00000000e+00, 2.32488486e-03, 1.70000000e-01]])

In [6]:
oid_gal   = pd.read_pickle('../data/oid_gal.pkl').object_id
oid_exgal = pd.read_pickle('../data/oid_exgal.pkl').object_id

In [7]:
def multi_weighted_logloss(y_true, y_pred, myweight=None):
    """
    @author olivier https://www.kaggle.com/ogrellier
    multi logloss for PLAsTiCC challenge
    """
    # class_weights taken from Giba's topic : https://www.kaggle.com/titericz
    # https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
    # with Kyle Boone's post https://www.kaggle.com/kyleboone
    classes = [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95]
    class_weight = {6: 1, 15: 2, 16: 1, 42: 1, 52: 1, 53: 1, 62: 1, 64: 2, 65: 1, 67: 1, 88: 1, 90: 1, 92: 1, 95: 1}
    if True:
        classes.append(99)
        class_weight[99] = 2
    
    if myweight is None:
        myweight = np.ones(M)
    y_p = y_pred * myweight
    
    # normalize
    y_p /= y_p.sum(1)[:,None]
    
    # Normalize rows and limit y_preds to 1e-15, 1-1e-15
    y_p = np.clip(a=y_p, a_min=1e-15, a_max=1 - 1e-15)
    # Transform to log
    y_p_log = np.log(y_p)
    # Get the log for ones, .values is used to drop the index of DataFrames
    # Exclude class 99 for now, since there is no class99 in the training set
    # we gave a special process for that class
    y_log_ones = np.sum(y_true * y_p_log, axis=0)
    # Get the number of positives for each class
    nb_pos = y_true.sum(axis=0).astype(float)
    # Weight average and divide by the number of positives
    class_arr = np.array([class_weight[k] for k in sorted(class_weight.keys())])
    y_w = y_log_ones * class_arr / nb_pos

    loss = - np.nansum(y_w) / np.sum(class_arr)
    return loss


def calc_gradient(f, X):
    """
    calc_gradient
    偏微分を行う関数
    関数fを変数xの各要素で偏微分した結果をベクトルにした勾配を返す
    
    @params
    f: 対象となる関数
    X: 関数fの引数のベクトル(numpy.array)
    
    @return
    gradient: 勾配(numpy.array)
    """
    
    h = 1e-4
    gradient = np.zeros_like(X)
    
    # 各変数についての偏微分を計算する
    for i in range(X.size):
        store_X = X[:]
        
        # f(x+h)
        X[i] += h
        f_x_plus_h = f(X)

        X = store_X[:]
        
        # f(x-h)
        X[i] -= h
        f_x_minus_h = f(X)
        
        # 偏微分
        gradient[i] = (f_x_plus_h - f_x_minus_h) / (2 * h)
        
    return gradient

def gradient_descent(f, X, learning_rate, max_iter, is_print=False):
    """
    gradient_descent
    最急降下法を行う関数
    
    @params
    f: 対象となる関数
    X: 関数fの引数のベクトル(numpy.array)
    learning_rate: 学習率
    max_iter: 繰り返し回数
    
    @return
    X: 関数の出力を最小にする(であろう)引数(numpy.array)
    """
    score_bk = 9999
    for i in range(max_iter):
        X -= (learning_rate * calc_gradient(f, X))
        score = f(X)
        if score_bk < score:
            break
        score_bk = score
        if is_print and i%50==0:
            print("[{:3d}] X = {}, f(X) = {:.7f}".format(i, X, score))
        
    return X

In [8]:
M = 15

X = np.ones(M)

In [9]:
di = {}
for i in np.arange(0.00, 1.01, 0.01):
    di[round(i, 2)] = np.append(np.ones(int(i*1000)), np.zeros( 1000-int(i*1000) ))

In [10]:
di[y_pred[0][4].round(2)].shape

(1000,)

In [16]:
def multi(idx):
    w = np.ones(15)
    y_true = np.array( [di[y_pred[idx][i].round(2)] for i in range(15)]).T
    y_pred_ = np.array([y_pred[idx] for i in range(1000)])
    f = lambda X: multi_weighted_logloss(y_true, y_pred_, w)
    gradient_descent(f, w, learning_rate=5, max_iter=100)
    return y_pred[idx] * w

In [136]:
idx = 3
w = np.ones(15)
y_true = np.array( [di[y_pred[idx][i].round(2)] for i in range(15)]).T
y_pred_ = np.array([y_pred[idx] for i in range(1000)])
f = lambda X: multi_weighted_logloss(y_true, y_pred_, w)
gradient_descent(f, w, learning_rate=2, max_iter=1000, is_print=True)

[  0] X = [1.         0.99918326 1.         0.88184764 1.02240508 1.
 0.91340641 1.10714602 1.         0.99052745 1.0515027  1.01673827
 1.         0.99892344 1.01829787], f(X) = 1.5492161


array([1.        , 0.9760716 , 1.        , 0.0113356 , 0.89108592,
       1.        , 0.16793319, 2.05966241, 1.        , 0.35447399,
       1.5737272 , 0.77293624, 1.        , 0.96845964, 0.46481887])

In [125]:
y_true.shape

(15,)

In [17]:
pool = Pool(cpu_count())
callback = pool.map(multi, range(9999))
pool.close()

In [141]:
arr = np.array(callback)
arr /= arr.sum(1)[:,None]
arr

array([[ 0.00000000e+00,  3.46044640e-04,  0.00000000e+00, ...,
         0.00000000e+00,  5.13883569e-03,  3.15934160e-01],
       [ 0.00000000e+00,  2.21632917e-02,  0.00000000e+00, ...,
         0.00000000e+00,  3.78950240e-03,  3.44748760e-01],
       [ 0.00000000e+00,  1.71575965e-02,  0.00000000e+00, ...,
         0.00000000e+00,  6.28044811e-03,  4.46907613e-01],
       ...,
       [ 0.00000000e+00,  2.72957664e-02,  0.00000000e+00, ...,
         0.00000000e+00,  2.89947263e-03,  1.23881745e-01],
       [ 0.00000000e+00,  1.35162954e-04,  0.00000000e+00, ...,
         0.00000000e+00,  3.21591011e-04,  5.77816686e-01],
       [ 0.00000000e+00,  6.32684433e-04,  0.00000000e+00, ...,
         0.00000000e+00, -2.26226519e-03,  2.87401572e-01]])

In [117]:
y_true.shape[1]

15

In [15]:
pool.close()

In [144]:
sub.shape

(3492890, 16)